# Librerias #

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
from math import radians, sin, cos, sqrt, atan2

# Load the restaurant dataset
restaurants_df = pd.read_csv("../data/df_comercios.csv")

## Revision de información ##

In [3]:
restaurants_df.head()

,id,name,rating,address,description,review,alias,image_url,is_closed,review_count,latitude,longitude,location,display_phone,price,distrito
0,GV-WXC3F4MUzwjhAH_f_XA,El Rincón Asturiano,1.0,"Calle de las Delicias, 26, 28045 Madrid, Spain",Spanish Asturian Tapas Bars,We are here for dinner again on two consecutiv...,el-rincón-asturiano-madrid-2,https://s3-media1.fl.yelpcdn.com/bphoto/cS74hs...,False,32,40.403985,-3.692258,"{'address1': 'Calle de las Delicias, 26', 'add...",+34 915 30 89 68,€€,Arganzuela
1,7zpK35tqV8uFtg9BGwfbRg,Donde da la Vuelta el Viento,5.0,"Calle de Mesón de Paredes, 81, 28012 Madrid, S...",Tapas Bars Spanish Modern European,Great place with friendly staff. I came for ta...,donde-da-la-vuelta-el-viento-madrid,https://s3-media2.fl.yelpcdn.com/bphoto/4YsbpQ...,False,37,40.406190,-3.701441,"{'address1': 'Calle de Mesón de Paredes, 81', ...",+34 910 17 72 40,€,Arganzuela
2,cxypKAKs_zzJ8kvB_6G2Bw,El Valle,4.0,"Calle de Sebastián Herrera, 6, 28012 Madrid, S...",Asturian Tapas Bars,The size of everything you order at this resta...,el-valle-madrid,https://s3-media2.fl.yelpcdn.com/bphoto/RApDyk...,False,21,40.403780,-3.699350,"{'address1': 'Calle de Sebastián Herrera, 6', ...",+34 914 67 70 07,€,Arganzuela
3,J6Mq8jWYD9ntHd0u4OQr9A,La Pequeña Graná,4.0,"Calle de Embajadores, 124, 28045 Madrid, Spain",Tapas Bars Tapas/Small Plates Beer Bar,Delicious tapas with awesome service. We order...,la-pequeña-graná-madrid-2,https://s3-media1.fl.yelpcdn.com/bphoto/BhD7tJ...,False,35,40.399390,-3.698510,"{'address1': 'Calle de Embajadores, 124', 'add...",+34 914 74 26 30,€,Arganzuela
4,om6h-4trsKlw9cOp53QXcg,Hermanos Egea,4.0,"Calle de la Batalla del Salado, 33, 28045 Madr...",Spanish Mediterranean Tapas/Small Plates,"The service was phenomenal, very understanding...",hermanos-egea-madrid-2,https://s3-media2.fl.yelpcdn.com/bphoto/rg5ZUM...,False,3,40.400160,-3.696130,"{'address1': 'Calle de la Batalla del Salado, ...",+34 914 68 46 82,€€,Arganzuela


# KNN y funciones de filtrado #
- El algoritmo Ball Tree es un método utilizado en el campo del aprendizaje automático y la búsqueda de vecinos más cercanos (k-nearest neighbors) para acelerar el cálculo de distancias entre puntos en espacios de alta dimensionalidad. El objetivo del algoritmo es construir una estructura de datos eficiente llamada Ball Tree que permite realizar consultas rápidas de vecinos más cercanos.

### Otras consideraciones ###

- Construcción del árbol: El proceso comienza seleccionando un punto cualquiera del conjunto de datos como "nodo raíz". Luego, se elige un punto central y se construyen dos subconjuntos de datos: uno que contiene puntos dentro de un radio específico (llamado "bola interior") y otro que contiene puntos fuera de ese radio (llamado "bola exterior"). Estos subconjuntos se convierten en los nodos hijos del nodo raíz. Este proceso se repite recursivamente para construir el árbol de forma jerárquica.

- Selección del punto central: Para construir los subconjuntos, se debe seleccionar un punto central. Una opción común es elegir el punto que maximice la distancia media a todos los demás puntos dentro de la bola exterior.

- División del espacio: Después de seleccionar el punto central, los puntos se asignan al subconjunto de la bola interior o la bola exterior según su distancia al punto central. La estructura del árbol se divide recursivamente en estos subconjuntos hasta que se cumpla algún criterio de terminación, como un número mínimo de puntos en cada nodo o un nivel máximo de profundidad.

- Búsqueda de vecinos más cercanos: Una vez que se ha construido el árbol, se puede utilizar para realizar consultas eficientes de vecinos más cercanos. Dado un punto de consulta, el algoritmo comienza en la raíz del árbol y examina recursivamente los nodos y subárboles según la distancia entre el punto de consulta y los puntos centrales de los nodos. Esto permite descartar rápidamente las regiones del árbol que no contienen vecinos más cercanos y enfocarse en las regiones relevantes.

In [4]:


def get_nearest_restaurants(user_latitude, user_longitude, k):
    #Creacion de modelo KNN
    knn = NearestNeighbors(n_neighbors=k, algorithm='ball_tree')
    
    #
    restaurant_coordinates = restaurants_df[['latitude', 'longitude']]
    knn.fit(restaurant_coordinates)
    
    #Encuentra los indices de los restaurantes mas cercanos al usuario
    _, indices = knn.kneighbors([[user_latitude, user_longitude]])
    
    #extrae los restaurantes mas cercanos del dataset
    nearest_restaurants = restaurants_df.iloc[indices[0]]
    
    return nearest_restaurants

def calculate_distance(lat1, lon1, lat2, lon2):
    #Convierte las coordenadas a radianes
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    #Formula haversine
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = 6371 * c  #Radio en kilometros

    return distance

def filter_restaurants(user_latitude, user_longitude, radius, k):
    #Obten el restaurante mas cercano al usuario
    nearest_restaurants = get_nearest_restaurants(user_latitude, user_longitude, k)
    
    #Filtrar los restaurantes que esten dentro del radio
    filtered_restaurants = []
    for _, row in nearest_restaurants.iterrows():
        distance = calculate_distance(
            user_latitude,
            user_longitude,
            row['latitude'],
            row['longitude']
        )
        if distance <= radius:
            filtered_restaurants.append(row)

    return pd.DataFrame(filtered_restaurants)

#Entradas
user_latitude = 40.421485858821995 #thebridge
user_longitude = -3.692703467811498  #thebridge
radius = 0.8  #radio en kilometros
k = 10 #retorno de cantidad de restaurantes por busqueda

nearby_restaurants = filter_restaurants(user_latitude, user_longitude, radius, k)

#imprime los restaurantes cercanos
for _, row in nearby_restaurants.iterrows():
    print(f"Restaurant: {row['name']} (Latitude: {row['latitude']}, Longitude: {row['longitude']})")


Restaurant: Café Gijón (Latitude: 40.42223, Longitude: -3.69233)
Restaurant: Tampu (Latitude: 40.42195, Longitude: -3.69347)
Restaurant: La Taberna del Gijón (Latitude: 40.42239, Longitude: -3.69263)
Restaurant: La Kitchen (Latitude: 40.4219775, Longitude: -3.6943897)
Restaurant: La Castafiore (Latitude: 40.4230906, Longitude: -3.6934631)
Restaurant: La Galette 2 (Latitude: 40.42336, Longitude: -3.69274)
Restaurant: New York Burger (Latitude: 40.4221, Longitude: -3.69082)
Restaurant: Pelotari (Latitude: 40.4221999, Longitude: -3.69071)
Restaurant: Faborit (Latitude: 40.4197818887255, Longitude: -3.6912852153182)
Restaurant: Al Mounia (Latitude: 40.4221517, Longitude: -3.690545)


/Users/me/anaconda3/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [5]:
from folium import plugins 
import folium


In [6]:
nearby_restaurants.head()

,id,name,rating,address,description,review,alias,image_url,is_closed,review_count,latitude,longitude,location,display_phone,price,distrito
923,J-ARIJinvKKzbmFWF65SWQ,Café Gijón,4.0,"Paseo de Recoletos, 21, 28004 Madrid, Spain",Coffee & Tea Spanish,"Allegedly, the oldest cafe in Madrid. I stoppe...",café-gijón-madrid-3,https://s3-media2.fl.yelpcdn.com/bphoto/eYbQ_r...,False,38,40.422230,-3.692330,"{'address1': 'Paseo de Recoletos, 21', 'addres...",+34 915 21 54 25,€€,Salamanca
920,giQvWtaDy24UfPhrkLKtlw,Tampu,5.0,"Calle de Prim, 13, 28004 Madrid, Spain",Peruvian Signature Cuisine Tapas Bars,I'm writing the review months after going so I...,tampu-madrid-2,https://s3-media2.fl.yelpcdn.com/bphoto/Swbj6_...,False,11,40.421950,-3.693470,"{'address1': 'Calle de Prim, 13', 'address2': ...",+34 915 64 19 13,€€€,Salamanca
933,8QqFzJmbmKLWqIfW5z36rA,La Taberna del Gijón,4.5,"Calle del Almirante. 30, 28004 Madrid, Spain",Tapas Bars Spanish,NaN,la-taberna-del-gijón-madrid-2,https://s3-media2.fl.yelpcdn.com/bphoto/uzWgxS...,False,3,40.422390,-3.692630,"{'address1': 'Calle del Almirante. 30', 'addre...",+34 915 21 10 55,€€,Salamanca
939,N44TgXe_KHDDn9E8tNx2yg,La Kitchen,2.0,"Calle de Prim, 5, 28004 Madrid, Spain",Mediterranean Modern European Tapas Bars,So good!! Excellent service!!! \nAsk for a d...,la-kitchen-madrid,https://s3-media1.fl.yelpcdn.com/bphoto/H161pQ...,False,14,40.421977,-3.694390,"{'address1': 'Calle de Prim, 5', 'address2': '...",+34 913 60 49 74,€€€,Salamanca
924,RS1bSQpdh9ltew6T5_F1Pg,La Castafiore,5.0,"Calle Marqués de Monasterio, 5, 28004 Madrid, ...",Spanish,Happy Valentine's Day! I hope whoever you lov...,la-castafiore-madrid,https://s3-media2.fl.yelpcdn.com/bphoto/NQiLY3...,False,8,40.423091,-3.693463,"{'address1': 'Calle Marqués de Monasterio, 5',...",+34 913 19 42 21,€€,Salamanca


In [7]:
map_madrid = folium.Map(location=[40.427919,-3.680877], zoom_start=14)

for (index, row) in nearby_restaurants.iterrows():
    folium.Marker(location = [row.loc["latitude"], row.loc["longitude"]],
    #popup = row.loc["rotulo"] + " " + row["desc_distrito_local"],
    tooltip = "click").add_to(map_madrid)

map_madrid